# **📓 Movie Review Sentiment Analysis using RoBERTa**

**📌 1. Environment Setup**

Install all required libraries for training and evaluating a RoBERTa-based sentiment classifier.

In [4]:
!pip install -q transformers torch tqdm pandas scikit-learn

**📌 2. Check Available Files**

Verify the working directory contents in the Colab environment.

In [5]:
!ls /content

sample_data  test.tsv  train.tsv


**📌 3. Load and Prepare Dataset**

Load the training dataset and extract only the required columns for sentiment classification.

In [6]:
import pandas as pd

train_df = pd.read_csv("/content/train.tsv", sep="\t")
train_df = train_df[["Phrase", "Sentiment"]]

**📌 4. Initialize Tokenizer**

Load the pre-trained RoBERTa tokenizer to convert text into token IDs.

In [7]:
from transformers import RobertaTokenizer

TOKENIZER = RobertaTokenizer.from_pretrained("roberta-base")
MAX_LEN = 128


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

**📌 5. Custom Dataset Class**

Create a PyTorch Dataset class to tokenize text and return model-ready tensors.

In [8]:
import torch
from torch.utils.data import Dataset

class SentimentDataset(Dataset):
    def __init__(self, df, tokenizer, max_len):
        self.texts = df["Phrase"].values
        self.labels = df["Sentiment"].values
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        enc = self.tokenizer(
            self.texts[idx],
            truncation=True,
            padding="max_length",
            max_length=self.max_len,
            return_tensors="pt"
        )
        return {
            "input_ids": enc["input_ids"].squeeze(0),
            "attention_mask": enc["attention_mask"].squeeze(0),
            "labels": torch.tensor(self.labels[idx], dtype=torch.long)
        }


**📌 6. Train-Validation Split**

Split the dataset into training and validation sets using stratified sampling.

In [9]:
from sklearn.model_selection import train_test_split

train_df, val_df = train_test_split(
    train_df,
    test_size=0.2,
    stratify=train_df["Sentiment"],
    random_state=42
)


**📌 7. DataLoaders**

Create DataLoader objects for efficient batch-wise training and evaluation.

In [10]:
from torch.utils.data import DataLoader

train_dataset = SentimentDataset(train_df, TOKENIZER, MAX_LEN)
val_dataset   = SentimentDataset(val_df, TOKENIZER, MAX_LEN)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader   = DataLoader(val_dataset, batch_size=16, shuffle=False)


**📌 8. Model Architecture**

Define a RoBERTa-based classification model with a dropout layer and linear head.

In [11]:
from transformers import RobertaModel

class RobertaClassifier(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.roberta = RobertaModel.from_pretrained("roberta-base")
        self.dropout = torch.nn.Dropout(0.3)
        self.classifier = torch.nn.Linear(768, 5)

    def forward(self, input_ids, attention_mask):
        out = self.roberta(
            input_ids=input_ids,
            attention_mask=attention_mask
        )
        pooled = out.last_hidden_state[:, 0]
        pooled = self.dropout(pooled)
        return self.classifier(pooled)


**📌 9. Training Setup**

Initialize model, loss function, optimizer, and device configuration.

In [12]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = RobertaClassifier().to(device)
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)


model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


**📌 10. Training Loop**

Train the model for one epoch and compute average training loss.

In [13]:
from tqdm import tqdm

def train_one_epoch(model, loader):
    model.train()
    total_loss = 0

    for batch in tqdm(loader):
        optimizer.zero_grad()

        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        outputs = model(input_ids, attention_mask)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    return total_loss / len(loader)


**📌 11. Validation Loop**

Evaluate model performance on validation data using accuracy.

In [14]:
def validate(model, loader):
    model.eval()
    correct, total = 0, 0

    with torch.no_grad():
        for batch in loader:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)

            outputs = model(input_ids, attention_mask)
            preds = torch.argmax(outputs, dim=1)

            correct += (preds == labels).sum().item()
            total += labels.size(0)

    return correct / total


## **📌 12. Model Training**

Train the model for multiple epochs and monitor validation accuracy.

In [15]:
EPOCHS = 2

for epoch in range(EPOCHS):
    loss = train_one_epoch(model, train_loader)
    acc  = validate(model, val_loader)

    print(f"Epoch {epoch+1}")
    print(f"Loss: {loss:.4f}")
    print(f"Val Accuracy: {acc:.4f}")


100%|██████████| 7803/7803 [46:06<00:00,  2.82it/s]


Epoch 1
Loss: 0.8198
Val Accuracy: 0.6904


100%|██████████| 7803/7803 [46:02<00:00,  2.82it/s]


Epoch 2
Loss: 0.7094
Val Accuracy: 0.6950


**📌 13. Save Model and Tokenizer**

Save trained model weights and tokenizer for inference and deployment.

In [16]:
import os

os.makedirs("/content/model", exist_ok=True)

torch.save(model.state_dict(), "/content/model/roberta_sentiment.pt")
TOKENIZER.save_pretrained("/content/model")


('/content/model/tokenizer_config.json',
 '/content/model/special_tokens_map.json',
 '/content/model/vocab.json',
 '/content/model/merges.txt',
 '/content/model/added_tokens.json')

**📌 14. Inference Function**

Define a function to predict sentiment and confidence for new input text.

In [17]:
def predict(text):
    model.eval()
    enc = TOKENIZER(
        text,
        return_tensors="pt",
        truncation=True,
        padding=True,
        max_length=128
    )
    input_ids = enc["input_ids"].to(device)
    attention_mask = enc["attention_mask"].to(device)

    with torch.no_grad():
        outputs = model(input_ids, attention_mask)
        probs = torch.softmax(outputs, dim=1)
        pred = torch.argmax(probs, dim=1).item()

    return pred, probs.cpu().numpy()


### **📌 15. Evaluation with Macro F1 Score**

Evaluate the model using both Accuracy and Macro F1 Score.

In [18]:
!pip install -q scikit-learn


In [19]:
from sklearn.metrics import f1_score
import numpy as np

def validate_with_f1(model, dataloader, device):
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for batch in dataloader:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)

            outputs = model(input_ids, attention_mask)
            preds = torch.argmax(outputs, dim=1)

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    all_preds = np.array(all_preds)
    all_labels = np.array(all_labels)

    accuracy = (all_preds == all_labels).mean()
    f1_macro = f1_score(all_labels, all_preds, average="macro")

    return accuracy, f1_macro


**📌 16. Final Evaluation Results**

In [20]:
acc, f1 = validate_with_f1(model, val_loader, device)

print(f"Validation Accuracy : {acc:.4f}")
print(f"Macro F1 Score      : {f1:.4f}")


Validation Accuracy : 0.6950
Macro F1 Score      : 0.6215


📌 17. Package Model for Deployment

In [22]:
!zip -r model.zip model



  adding: model/ (stored 0%)
  adding: model/special_tokens_map.json (deflated 84%)
  adding: model/tokenizer_config.json (deflated 76%)
  adding: model/merges.txt (deflated 53%)
  adding: model/roberta_sentiment.pt (deflated 12%)
  adding: model/vocab.json (deflated 68%)


## **🏁 Final Note**

This notebook documents the complete training and evaluation pipeline for a RoBERTa-based sentiment analysis model, which is later deployed using Streamlit as an interactive web application.

In [23]:
!ls -lh /content


total 431M
drwxr-xr-x 2 root root 4.0K Dec 13 21:22 model
-rw-r--r-- 1 root root 420M Dec 13 21:47 model.zip
drwxr-xr-x 1 root root 4.0K Dec  9 14:42 sample_data
-rw-r--r-- 1 root root 3.3M Dec 13 19:42 test.tsv
-rw-r--r-- 1 root root 8.1M Dec 13 19:42 train.tsv


In [26]:
!cd /content/model && split -b 100M roberta_sentiment.pt roberta_part_


In [27]:
!ls -lh /content/model


total 953M
-rw-r--r-- 1 root root 446K Dec 13 21:22 merges.txt
-rw-r--r-- 1 root root 100M Dec 13 22:08 roberta_part_aa
-rw-r--r-- 1 root root 100M Dec 13 22:08 roberta_part_ab
-rw-r--r-- 1 root root 100M Dec 13 22:08 roberta_part_ac
-rw-r--r-- 1 root root 100M Dec 13 22:08 roberta_part_ad
-rw-r--r-- 1 root root  76M Dec 13 22:08 roberta_part_ae
-rw-r--r-- 1 root root 476M Dec 13 21:22 roberta_sentiment.pt
-rw-r--r-- 1 root root  958 Dec 13 21:22 special_tokens_map.json
-rw-r--r-- 1 root root 1.2K Dec 13 21:22 tokenizer_config.json
-rw-r--r-- 1 root root 976K Dec 13 21:22 vocab.json


In [28]:
import torch
import gc

gc.collect()
torch.cuda.empty_cache()


In [29]:
# Convert model to half precision (FP16)
model_fp16 = model.half()

# Save smaller model
torch.save(model_fp16.state_dict(), "/content/roberta_sentiment_fp16.pt")

print("FP16 model saved")


FP16 model saved


In [30]:
!ls -lh /content/roberta_sentiment_fp16.pt


-rw-r--r-- 1 root root 238M Dec 13 22:14 /content/roberta_sentiment_fp16.pt
